In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
encoding = "cp1252"
encoding1 = "ISO-8859-1"
df = pd.read_csv('../mi-person/data/sentiment-140-training.1600000.processed.noemoticon.csv', encoding=encoding)

X_train = df[:10000]
y_train = df[1589999:]
X_test = df[10001:20001]
y_test = df[30001:40001]

df.sample(7000)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
666163,0,2245183318,Fri Jun 19 15:46:24 PDT 2009,NO_QUERY,Seductiionn_Ox,So Whats New ? I Died My Hair Last Niqht Im No...
1426874,4,2059298361,Sat Jun 06 16:39:01 PDT 2009,NO_QUERY,JoyEWil,@MatthewSantos Definitely root for the Cubbies!
782237,0,2323608250,Thu Jun 25 01:02:43 PDT 2009,NO_QUERY,dellvink,Sad to hear our tailor in Beijing has closed t...
699587,0,2254492614,Sat Jun 20 09:50:01 PDT 2009,NO_QUERY,JustinCampbelll,Beingng Lazy &amp; Hungover.....Back Into Wor ...
145188,0,1882153330,Fri May 22 06:20:13 PDT 2009,NO_QUERY,zorp75ck,"@wilborne Nope, they are the same age that the..."
...,...,...,...,...,...,...
642467,0,2235508642,Fri Jun 19 01:22:54 PDT 2009,NO_QUERY,Zaraist,How the hell am i suposed to get Twig Guy's se...
16674,0,1556009061,Sat Apr 18 21:21:17 PDT 2009,NO_QUERY,pcp071098,@liquilife Ooops.. I had a bug But I fixed ...
576380,0,2211600840,Wed Jun 17 13:03:14 PDT 2009,NO_QUERY,Stephanya,@coldfusion1970 I still have a year on my cont...
146791,0,1882580430,Fri May 22 07:07:35 PDT 2009,NO_QUERY,djcraze90,Taking a rest from driving at a citgo. Kristin...


In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 6)
(10000, 6)
(10000, 6)
(10000, 6)


In [4]:
df.columns = ["target", "ids", "date", "flag", "user", "text"]


In [8]:
df.head()
df["target"].value_counts()

4    800000
0    799999
Name: target, dtype: int64

In [6]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tatchiwiggers/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
sentiments = SentimentIntensityAnalyzer()
df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["text"]]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["text"]]
df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["text"]]
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["text"]]
df.head()

KeyboardInterrupt: 

In [ ]:
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05:
        sentiment.append('Positive')
    elif i <= -0.05:
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.head()

In [ ]:
df["Sentiment"].value_counts()

In [ ]:
# RegEx for removing non-letter characters
import re

# NLTK library for the remaining steps
import nltk
nltk.download("stopwords")   # download list of stopwords (only once; need not run it again)
from nltk.corpus import stopwords # import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tatchiwiggers/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def msg_to_words(string):
    """Convert string into a sequence of words."""

    string = re.sub(r"[^a-zA-Z0-9]", " ", string.lower())
    words = string.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]

    # Return final list of words
    return words

In [ ]:
msg_to_words("""@joanneteh We all have grumpiness. New computer = working""")

['joanneteh', 'grumpi', 'new', 'comput', 'work']

In [ ]:
import os
import pickle

cache_dir = os.path.join("cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = list(map(msg_to_words, data_train))
        words_test = list(map(msg_to_words, data_test))
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test


# Preprocess data
words_train, words_test, labels_train, labels_test = preprocess_data(
        data_train, data_test, labels_train, labels_test)

# Take a look at a sample
print("\n--- Raw review ---")
print(data_train[1])
print("\n--- Preprocessed words ---")
print(words_train[1])
print("\n--- Label ---")
print(labels_train[1])

NameError: name 'data_train' is not defined